In [22]:
import wandb
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
plt.style.use('seaborn')

api = wandb.Api(timeout=30)
entity = "joasiee"
optims = ["GOMEA", "AdaptiveStochasticGradientDescent"]

def get_runs(project, filters={}):
    return api.runs(entity + "/" + project, filters=filters)

def get_finished_run_data(runs):
    names, configs, histories = [], [], []
    for run in runs:
        if run.state == "finished":
            names.append(run.name)
            configs.append(
                {k: v for k, v in run.config.items()
                 if not k.startswith('_')})
            history = run.scan_history()
            histories.append(pd.DataFrame.from_dict(history))
    return names, configs, histories

def run_to_dict(name, config, history):
    res = {"name": name, "config": config, "resolutions": []}
    
    nr_resolutions = int(config["NumberOfResolutions"])
    for r in range(0, nr_resolutions):
        condition = ~np.isnan(history[f"R{r}/metric"]) if nr_resolutions > 1 else history.index
        indices = history.index[condition]
        columns = ["_step", "_runtime", "_timestamp"] +  [c for c in history.columns if f"R{r}/" in c]
        history_r = history[columns]
        res["resolutions"].append(history_r.loc[indices].to_dict())
    
    return res

In [23]:
names, configs, histories = get_finished_run_data(get_runs("zandbak"))

In [27]:
histories[0]["R0/time[ms]"]

0      92.2
1      59.1
2      79.5
3     108.2
4      38.2
5      29.9
6        39
7      75.1
8      47.0
9      31.9
10     35.4
11     32.0
12     31.4
13     32.7
14     30.2
15     30.2
16      NaN
17     30.5
18     30.7
19     32.0
20     30.8
21     36.7
22     29.9
23     29.1
24      NaN
25     30.2
26     28.8
27     32.1
28     29.7
29     29.4
30     31.2
31     27.9
32     28.4
33     34.3
34     30.1
35     30.7
36      NaN
37     27.0
38     34.1
39     28.9
40     29.0
41     35.9
42     27.3
43     27.2
44     31.5
45     26.3
46     26.8
47     34.3
48     28.7
49     26.7
Name: R0/time[ms], dtype: object